# Function calling

In this example, we will use function calling (tools) to obtain information from a Git repository.

## Dependencies

We will use LangChain again and we also need GitPython to interact with a Git repository.

In [19]:
import sys
!{sys.executable} -m pip install GitPython langchain-openai langchain-core

## Model

We will use an OpenAI model again, so we need an API key.

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


## Defining the functions

Let's define the functions (tools) that can be called to fulfill a task sent to the model.

It is important to give informative names for the function and the parameters and provide some form of documentation. These details will be provided in the context of the query to the model.

In [2]:
from datetime import date
from typing import List
from langchain_core.tools import tool
from git import Git

repo_directory = './defects4j'

@tool
def list_commits_between(start_date: date, end_date: date) -> str:
    """ Returns the messages of the commits from an interval between dates.
    Args:
        start_date: The start of the interval.
        end_date: The end of the date.
    """
    repo = Git(repo_directory)
    logs = repo.log('--since={}'.format(start_date), '--until={}'.format(end_date), '--oneline')
    return logs

## Binding the functions to the model

Now, let's register the functions into the model. After that, the calls to the model will inform about the existence of the functions, what they can do and their parameters.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools([list_commits_between])

## Querying the model

Let's ask the model to summarize what was done in the project in the last year. 

To do so, we have to tell the model the date of today (do you want to try without doing so?)).

In [33]:
from datetime import date

query = '''Summarize what has been done in the project last calendar year. Today is {}. 
           In your answer, provide the dates of which your results refer to.'''.format(date.today())

Now let's query the model:

In [34]:
msg_with_tool_calls = llm_with_tools.invoke(query)

Let's inspect the response.

In [35]:
msg_with_tool_calls

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZVfHQOBk4Y94aNpgg4l1mvlG', 'function': {'arguments': '{"start_date":"2024-01-01","end_date":"2024-12-31"}', 'name': 'list_commits_between'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 130, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-92c41174-9bbf-4e76-9f30-44a252ecf403-0', tool_calls=[{'name': 'list_commits_between', 'args': {'start_date': '2024-01-01', 'end_date': '2024-12-31'}, 'id': 'call_ZVfHQOBk4Y94aNpgg4l1mvlG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_tokens': 33, 'total_tokens': 163, 'input_token_

Now, we have to pass the previous messages to the model. Let's prepare the list of messages.

In [36]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query), msg_with_tool_calls]

for tool_call in msg_with_tool_calls.tool_calls:
    selected_tool = locals()[tool_call['name']]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    tool_msg.pretty_print()

================================= Tool Message =================================
Name: list_commits_between

6d54320e Update version number.
8bb959b6 Consistent target JVM version for all defect builds (#628)
d062ed66 Update Dockerfile to Java 11 (#622)
6a99ea10 Add randoopBloodhound to supported test generators
640a67c2 Update README.
cfb51a7c Java 11 compatibility (#613)
d527ffd0 docs: update README.md (#610)
089af8d0 Add convenience script for test stats.
ac9c0037 Update Randoop to 4.3.3
ff93e4f8 Reorganize notes about `cpanm` (#590)
45f22d94 Revert "add corrected version of Camille's Bloodhound script" randoop releases need to add jacocoagent.jar
1edb002f add corrected version of Camille's Bloodhound script
0e238102 Be more explicit about Subversion
e85036c0 How to determine the method names of changed methods (#582)
796a1bdb Improve documentation of `tests.trigger.cause` (#581)
ed34738c Sort lines
f8ad2018 Context about different command-line interfaces (#544)
90d4ecd7 Use actions

Now, it is time to send to the model and see the answer.

In [37]:
result = llm_with_tools.invoke(messages)
result.pretty_print()

================================== Ai Message ==================================

From January 1, 2024, to December 31, 2024, several significant updates and changes were made to the project. Here is a summary of the key activities:

1. **Version Updates**:
   - The project version number was updated.
   - Java 11 compatibility was introduced and documented.

2. **Dockerfile Adjustments**:
   - The Dockerfile was updated to use Java 11.

3. **Testing Enhancements**:
   - The project added support for a new test generator, RandoopBloodhound.
   - Randoop was updated to version 4.3.3.

4. **Documentation Improvements**:
   - The README file was updated to reflect new features and compatibility.
   - Additional documentation was provided for the `tests.trigger.cause` to improve understanding.
   - Clarifications and examples were added to the README.md.

5. **Build Improvements**:
   - Consistent JVM targeting was established for all defect builds.
   - A convenience script for test stati

## Exercise

1. Modify the code to ask about a specific contributor.